In [28]:
# from urllib import urlopen
from urllib.request import urlopen
import re
from pprint import pprint
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import csv
import io

In [203]:
place_type = "pub"  # Choose: restaurant, cafe, coffee, pub
URL = "https://restoran.kz/"+place_type
df = place_type+'1'+'.csv'
HEADERS = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36',
           'accept': '*/*'}

In [48]:
def get_html(url, params=None):
    r = requests.get(url, headers=HEADERS, params=params)
    return r

In [194]:
def getWorkTime(link):
    pattern_1='<div class="place-info-list__item"><span class="place-info-list__icon">⌚️<\/span><span class="place-info-list__text">(.{1,}?)<\/span><\/div>'
    try:
        with urllib.request.urlopen(link) as response:
            html = response.read().decode('utf-8')
        oo=re.findall(pattern_1, html)
    except urllib.error.HTTPError:
        oo="Не указано"
    return oo

In [120]:
def get_content(html):
    soup = bs(html, 'html.parser')
    places = soup.find_all('div', class_='place-list-card__body')

    arr = []
    for place in places:
        reserv = place.find('a', class_='btn btn-primary')
        if reserv:
            reserv = 1
        else:
            reserv = 0

        reviews = place.find('a', class_='text-small')
        if reviews:
            reviews = reviews.get_text(strip=True)
        else:
            reviews = 'No reviews'

        extra = place.find('div', class_='list-unstyled mb-4').find_next('li').find_next('li').find_next('li')
        if extra:
            extra = extra.get_text(strip = True)
        else:
            extra = 'No extra'

        cuisine = place.find('div', class_='list-unstyled mb-4').find_next('li')
        if cuisine:
            cuisine = cuisine.get_text(strip=True)
        else:
            cuisine = 'No cuisine'

        address = place.find('p', class_='place-list-card__subtitle')
        if address:
            address = address.get_text(strip=True)
        else:
            address = 'No address information'

        price = place.find('div', class_='list-unstyled mb-4').find_next('li').find_next('li')
        if price:
            price = price.get_text(strip=True)
        else:
            price = 'No price information'

        inside_link = "https://restoran.kz" + place.find('a', class_='link-inherit-color').get('href')
        if (getWorkTime(inside_link)):
            time = getWorkTime(inside_link)
        else:
            time = 'Не указано'

        arr.append({
            'Name': place.find('a', class_='link-inherit-color').get_text(strip=True),
            'Type': place_type,
            'Address': address,
            'Cuisine type': cuisine,
            'Price': price,
            'Extra': extra,
            'Reservation': reserv,
            'Number of reviews': reviews,
            'Working Hours': time
            # 'Rating': place.find('span', class_ = 'rating-stars rating-stars_star_0')['aria-label']
        })
    return arr

In [121]:
def to_csv(items, path):
    with open(path, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file, delimiter=';')
        writer.writerow(['Name', 'Type', 'Address', 'Cuisine type', 'Price', 'Extra', 'Reservation', 'Number of reviews', 'Working Hours'])
        for item in items:
            writer.writerow([item['Name'], item['Type'], item['Address'], item['Cuisine type'],
                             item['Price'], item['Extra'], item['Reservation'], item['Number of reviews'], item['Working Hours']])

In [178]:
def parse():
    html = get_html(URL)
    if html.status_code == 200:
        arr = []
        errors = []
        # change range parameter with corresponding maximum page:
        # 29 - restaurants, 40 - cafe, 12 - coffee shop, 16 - pubs
        if place_type == "restaurant":
            pages = 29
        elif place_type == "cafe":
            pages = 40
        elif place_type == "coffee":
            pages = 12
        elif place_type == "pub":
            pages = 16

        for page in range(1, pages+1):
            print(f'Parsing of page number {page}')
            html = get_html(URL, params={'page': page})
            arr.extend(get_content(html.text))
        to_csv(arr, df)
        # print(arr)
        # print(f'Number of parsed restaurants: {len(arr)}')
    else:
        print("Wrong url")

In [204]:
parse()

Parsing of page number 1
Parsing of page number 2
Parsing of page number 3
Parsing of page number 4
Parsing of page number 5
Parsing of page number 6
Parsing of page number 7
Parsing of page number 8
Parsing of page number 9
Parsing of page number 10
Parsing of page number 11
Parsing of page number 12
Parsing of page number 13
Parsing of page number 14
Parsing of page number 15
Parsing of page number 16
